In [2]:
##importing libraries
import torch
import torch.optim as optim
import torch.nn as nn
import os
from torch.utils.data import Dataset, DataLoader , random_split
from torchvision import transforms , datasets
from PIL import Image
import pandas as pd
import re
import random
import shutil
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import numpy as np


/home/jaskaran/.conda/envs/training/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# custom class to process dataset
class GeneticArrayDataset(Dataset):
    def __init__(self,root_folder, transform =None):
        self.root_folder = root_folder
        self.transform = transform
        self.class_labels = sorted(os.listdir(root_folder))
        self.class_to_idx = {label: idx for idx ,label in enumerate(self.class_labels)}
        self.file_list = self._build_file_list()
    
    def _build_file_list(self):
        file_list = []
        for class_label in self.class_labels:
            class_path = os.path.join(self.root_folder,class_label)
            class_array = [os.path.join(class_path,array_path) for array_path in os.listdir(class_path)]
            file_list.extend(class_array)
        return file_list
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self,index):
        file_path = self.file_list[index]
        array = np.loadtxt(file_path)
        mean = np.mean(array)
        std = np.std(array)
        array = (array - mean) / std ##normalized array

        class_label = os.path.basename(os.path.dirname(file_path))
        label = self.class_to_idx[class_label]
        find = re.match(r".+?[_].+?[_].+?[_](.+?)[.][t][x][t]",file_path)
        array = torch.from_numpy(array)
        return {'array': array, 'label': label, "coefficient":find[1]}

In [5]:
## Loading the data
data = GeneticArrayDataset("/home/jaskaran/data_arrays")
generator = torch.Generator().manual_seed(46)
length_data = len(data)
training_size = int(0.8*length_data)
testing_size = length_data - training_size

train_data , test_data = random_split(data,[training_size, testing_size],generator=generator)


training_dataloader= DataLoader(train_data,batch_size=64,shuffle=True, drop_last=True)
testing_dataloader = DataLoader(test_data,batch_size=64,shuffle=True, drop_last=True)

In [31]:
##Defining the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,
                               out_channels = 32,
                               kernel_size = 9,
                               stride = 1,
                               padding = 9)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)
        self.flatten = nn.Flatten(start_dim=1)
        self.fc1 = nn.Linear(96800,256)
        self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,2)

    def forward(self,x):
        x = x.float()
        
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
            
        x = self.fc3(x)
        return x
         

In [32]:
## Training the CNN model
torch.manual_seed(45)
model = CNN()
model.to("cuda:2")

if torch.cuda.is_available():
    torch.cuda.manual_seed(45)
    torch.cuda.manual_seed_all(45)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.0001)

num_epoch = 20

for epoch in range(num_epoch):

    model.train()
    total_loss = 0.0

    for batch in training_dataloader:
        arrays = batch["array"]
        arrays = arrays.unsqueeze(1)
        arrays = arrays.to("cuda:2")
        labels  = batch["label"].to("cuda:2")
        optimizer.zero_grad()
        output = model(arrays)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * arrays.size(0)  # Multiply by batch size

    # Calculate average loss over all batches
    average_loss = total_loss / len(training_dataloader.dataset)
    
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for batch in testing_dataloader:
            arrays = batch["array"]
            arrays = arrays.unsqueeze(1)
            arrays = arrays.to("cuda:2")
            labels = batch["label"].to("cuda:2")
            output = model(arrays)
            _ , predicted = torch.max(output.data,1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct/total
    print(f"Epoch : {epoch} , Loss : {average_loss:.2f} , Accuracy : {accuracy:.2%}")

print("Model is trained")




Epoch : 0 , Loss : 0.10 , Accuracy : 98.44%
Epoch : 1 , Loss : 0.07 , Accuracy : 98.54%
Epoch : 2 , Loss : 0.05 , Accuracy : 98.64%
Epoch : 3 , Loss : 0.04 , Accuracy : 98.44%
Epoch : 4 , Loss : 0.03 , Accuracy : 98.54%
Epoch : 5 , Loss : 0.02 , Accuracy : 98.69%
Epoch : 6 , Loss : 0.01 , Accuracy : 98.54%
Epoch : 7 , Loss : 0.01 , Accuracy : 98.54%
Epoch : 8 , Loss : 0.01 , Accuracy : 98.54%
Epoch : 9 , Loss : 0.01 , Accuracy : 98.49%
Epoch : 10 , Loss : 0.00 , Accuracy : 98.59%
Epoch : 11 , Loss : 0.00 , Accuracy : 98.69%
Epoch : 12 , Loss : 0.00 , Accuracy : 98.59%
Epoch : 13 , Loss : 0.00 , Accuracy : 98.69%
Epoch : 14 , Loss : 0.00 , Accuracy : 98.59%
Epoch : 15 , Loss : 0.00 , Accuracy : 98.54%
Epoch : 16 , Loss : 0.00 , Accuracy : 98.74%
Epoch : 17 , Loss : 0.00 , Accuracy : 98.69%
Epoch : 18 , Loss : 0.01 , Accuracy : 98.54%
Epoch : 19 , Loss : 0.03 , Accuracy : 98.39%
Model is trained


In [33]:
### Savong the model dict
torch.save(model.state_dict(), f'model_state_dict_array_filtering_lr0.0001_32_deep_leanrning_drop_0.2_9*9.pth')